In [ ]:
[2024-11-03, 19:13:39 UTC] {logging_mixin.py:190} INFO - tweets_tweets_tweets :[<Tweet id="1680289725494730752">, <Tweet id="1827573383124439550">, <Tweet id="1816830681378332972">, <Tweet id="1678515599746506754">, <Tweet id="1851218767780704306">, <Tweet id="1671238803468500992">, <Tweet id="1719637715468046469">, <Tweet id="1679946016806834176">, <Tweet id="1755262588483698957">, <Tweet id="1823057947859251625">, <Tweet id="1836340621851885869">, <Tweet id="1827551868580487321">, <Tweet id="1844390884370215138">, <Tweet id="1791480027416060350">, <Tweet id="1772856319701053581">, <Tweet id="1823251550502035695">, <Tweet id="1699514130225406258">, <Tweet id="1851224940957351972">, <Tweet id="1804192333082300758">, <Tweet id="1801008261090484510">]; type: <class 'list'>


In [ ]:
volumes:
  postgres-db-volume:

services:
  postgres:
    image: postgres:13
    volumes:
      - postgres-db-volume:/var/lib/postgresql/data
    healthcheck:
      test: ["CMD", "pg_isready", "-U", "airflow"]
      interval: 5s
      timeout: 5s
      retries: 5
    user: root
    env_file:
      - ./.env

  redis:
    image: redis:latest
    ports:
      - "6379:6379"

  airflow-init:
    image: apache/airflow:2.7.1
    container_name: airflow_initdb
    command: start_initdb
    depends_on:
      - postgres
    env_file:
      - ./.env


  airflow-webserver:
    command: start_server
    build:
      context: .
      dockerfile: ./Dockerfile
    volumes:
      - ./airflow/dags:/opt/airflow/dags
      - ./airflow/logs:/opt/airflow/logs
      - ./airflow/plugins:/opt/airflow/plugins
    ports:
      - "8080:8080"
    depends_on:
      postgres:
        condition: service_healthy
      airflow-init: 
        condition: service_completed_successfully
    env_file:
      - ./.env
    # command: ["webserver"]

  airflow-scheduler:
    image: apache/airflow:2.7.1
    container_name: airflow_scheduler
    command: start_scheduler
    volumes:
      - ./airflow/dags:/opt/airflow/dags
      - ./airflow/logs:/opt/airflow/logs
      - ./airflow/plugins:/opt/airflow/plugins
    depends_on:
      postgres:
        condition: service_healthy
      airflow-init: 
        condition: service_completed_successfully
    env_file:
      - ./.env
    # command: ["scheduler"]

  docs:
    build:
      context: .
      dockerfile: ./docs/Dockerfile
    image: pipeline_etl_docs
    container_name: pipeline_etl_docs
    volumes:
      - .:/app/:z
    restart: always
    ports:
      - "9021:9000"
    env_file:
      - ./.env
    command: start_docs


In [ ]:
volumes:
  postgres-db-volume:

services:
  postgres:
    image: postgres:13
    volumes:
      - postgres-db-volume:/var/lib/postgresql/data
    healthcheck:
      test: ["CMD", "pg_isready", "-U", "airflow"]
      interval: 5s
      timeout: 5s
      retries: 5
    user: root
    env_file:
      - ./.env

  redis:
    image: redis:latest
    ports:
      - "6379:6379"

  airflow-init:
    image: apache/airflow:2.7.1-python3.9
    depends_on:
      - postgres
    command: >
      bash -c "
      airflow db migrate && 
      airflow users create --username admin --firstname Admin --lastname User --role Admin --email admin@example.com --password admin"
    env_file:
      - ./.env

  airflow-webserver:
    image: apache/airflow:2.7.1-python3.9
    build:
      context: .
      dockerfile: ./Dockerfile
    volumes:
      - ./airflow/dags:/opt/airflow/dags
      - ./airflow/logs:/opt/airflow/logs
      - ./airflow/plugins:/opt/airflow/plugins
    ports:
      - "8080:8080"
    depends_on:
      postgres:
        condition: service_healthy
      airflow-init: 
        condition: service_completed_successfully
    env_file:
      - ./.env
    command: ["webserver"]

  airflow-scheduler:
    image: apache/airflow:2.7.1-python3.9
    build:
      context: .
      dockerfile: ./Dockerfile
    volumes:
      - ./airflow/dags:/opt/airflow/dags
      - ./airflow/logs:/opt/airflow/logs
      - ./airflow/plugins:/opt/airflow/plugins
    depends_on:
      postgres:
        condition: service_healthy
      airflow-init: 
        condition: service_completed_successfully
    env_file:
      - ./.env
    command: ["scheduler"]

  docs:
    build:
      context: .
      dockerfile: ./docs/Dockerfile
    image: pipeline_etl_docs
    container_name: pipeline_etl_docs
    volumes:
      - .:/app/:z
    restart: always
    ports:
      - "9021:9000"
    env_file:
      - ./.env
    command: mkdocs serve -a 0.0.0.0:9000
    # command: mkdocs serve -a 0.0.0.0:9000


In [5]:
import os

In [6]:
os.chdir('airflow/plugins/scrape_test/')

In [7]:
pwd

"c:\\Users\\kaisensdata\\Music\\AIRFLOW\\Pipeline de Données Twitter pour l'Analyse de Sentiments\\pipeline_etl\\airflow\\plugins\\scrape_test"

In [10]:
pip install twikit==2.1.3

    extract-msg (<=0.29.*)
                 ~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 24.3
[notice] To update, run: C:\Users\kaisensdata\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
from scraper import TwitterScraper
import asyncio
scraper = TwitterScraper()
await scraper.main()


TypeError: AsyncClient.__init__() got an unexpected keyword argument 'proxy'

In [14]:
pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.


    extract-msg (<=0.29.*)
                 ~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 24.3
[notice] To update, run: C:\Users\kaisensdata\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
import nest_asyncio
import asyncio
from scraper import TwitterScraper

# Appliquer nest_asyncio pour permettre l'exécution dans un notebook
nest_asyncio.apply()

# Créer une instance de TwitterScraper
scrape = TwitterScraper()

# Exécuter la méthode main()
asyncio.run(scrape.main())


TypeError: AsyncClient.__init__() got an unexpected keyword argument 'proxy'

In [2]:
import os
os.chdir('airflow/plugins/scrape_test/')

In [ ]:
from twikit import Client, TooManyRequests
import asyncio
from datetime import datetime
import csv
from configparser import ConfigParser
from random import randint
import os

class TwitterScraper:
    def __init__(self, logger=None):
        self.logger = logger  # Utilise le logger d'Airflow
        self.minimum_tweets = int(os.getenv('MINIMUM_TWEETS', 100))
        self.query = os.getenv('QUERY')

    async def get_tweets(self, client, tweets):
        if tweets is None:
            self.logger.info(f'{datetime.now()} - Getting initial tweets...')
            tweets = await client.search_tweet(self.query, product='Latest')
        else:
            wait_time = randint(5, 10)
            self.logger.info(f'{datetime.now()} - Waiting {wait_time} seconds before fetching more tweets...')
            await asyncio.sleep(wait_time)
            tweets = await tweets.next()
        return tweets

    async def main(self):
        # Chargement des informations d'identification
        config = ConfigParser()
        config.read('config.ini')

        # Vérification de la présence de la section 'X'
        if not config.has_section('X'):
            self.logger.error("La section 'X' n'existe pas dans la configuration.")
            raise ValueError("Configuration invalide : section 'X' manquante.")
        
        try:
            username = config['X']['username']
            email = config['X']['email']
            password = config['X']['password']
        except KeyError as e:
            self.logger.error(f"Clé manquante dans la configuration : {e}")
            raise

        # Nom du fichier CSV avec horodatage
        csv_filename = f"tweets_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        
        # Initialisation du client Twitter
        client = Client(language='en-US')
        cookies_file = 'cookies.json'

        # Gestion des cookies
        if os.path.exists(cookies_file):
            try:
                self.logger.info(f'{datetime.now()} - Loading cookies from {cookies_file}')
                client.load_cookies(cookies_file)
            except Exception as e:
                self.logger.warning(f'{datetime.now()} - Failed to load cookies: {e}. Re-authenticating...')
                await client.login(auth_info_1=username, auth_info_2=email, password=password)
                client.save_cookies(cookies_file)
        else:
            self.logger.info(f'{datetime.now()} - No cookies found, logging in...')
            await client.login(auth_info_1=username, auth_info_2=email, password=password)
            client.save_cookies(cookies_file)
            self.logger.info(f'{datetime.now()} - Cookies saved to {cookies_file}')

        # Préparation du fichier CSV
        with open(csv_filename, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Tweet_count', 'Username', 'Text', 'Created At', 'Retweets', 'Likes'])

        # Récupération des tweets
        tweet_count = 0
        tweets = None

        while tweet_count < self.minimum_tweets:
            try:
                tweets = await self.get_tweets(client, tweets)
            except TooManyRequests as e:
                rate_limit_reset = datetime.fromtimestamp(e.rate_limit_reset)
                self.logger.warning(f'{datetime.now()} - Rate limit reached. Waiting until {rate_limit_reset}')
                wait_time = rate_limit_reset - datetime.now()
                await asyncio.sleep(wait_time.total_seconds())
                continue

            if not tweets:
                self.logger.info(f'{datetime.now()} - No more tweets found')
                break

            for tweet in tweets:
                tweet_count += 1
                tweet_data = [
                    tweet_count, tweet.user.name, tweet.text, tweet.created_at, 
                    tweet.retweet_count, tweet.favorite_count
                ]

                # Écriture des données dans le fichier CSV
                with open(csv_filename, 'a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow(tweet_data)

                # Log chaque batch de tweets pour le suivi
                self.logger.info(f'{datetime.now()} - Collected {tweet_count} tweets so far')

        self.logger.info(f'{datetime.now()} - Scraping completed! Total tweets collected: {tweet_count}')



# from twikit import Client, TooManyRequests
# import time
# from datetime import datetime
# import csv
# from configparser import ConfigParser
# from random import randint
# import os
# import logging
# # from config import logger
# # from config import config
# # twitterscraper_log = logger.Logger(config.# twitterscraper_log, "DEBUG").logger

# class TwitterScraper:
#     def __init__(self, logger=None):
#         self.logger = logger or logging.getLogger(__name__)
#         self.minimum_tweets = int(os.getenv('MINIMUM_TWEETS', 100))
#         self.query = os.getenv('QUERY')

#     async def get_tweets(self, client, tweets):
#         if tweets is None:
#             self.logger.info(f'{datetime.now()} - Getting tweets...')
#             tweets = await client.search_tweet(self.query, product='Latest')
#         else:
#             wait_time = randint(5, 10)
#             self.logger.info(f'{datetime.now()} - Getting next tweets after {wait_time} seconds...')
#             # twitterscraper_log.info(f'{datetime.now()} - Getting next tweets after {wait_time} seconds...')
#             time.sleep(wait_time)
#             tweets = await tweets.next()
#         return tweets

#     async def main(self):
#         # Load login credentials
#         config = ConfigParser()
#         config.read('config.ini')
#         username = config['X']['username']
#         email = config['X']['email']
#         password = config['X']['password']

#         with open('tweets.csv', 'w', newline='', encoding='utf-8') as file:
#             writer = csv.writer(file)
#             writer.writerow(['Tweet_count', 'Username', 'Text', 'Created At', 'Retweets', 'Likes'])

#         client = Client(language='en-US')
#         cookies_file = 'cookies.json'
#         if os.path.exists(cookies_file):
#             self.logger.info(f'{datetime.now()} - Loading cookies from {cookies_file}')
#             # twitterscraper_log.info(f'{datetime.now()} - Loading cookies from {cookies_file}')
#             client.load_cookies(cookies_file)
#         else:
#             self.logger.info(f'{datetime.now()} - No cookies found, logging in...')
#             # twitterscraper_log.info(f'{datetime.now()} - No cookies found, logging in...')
#             await client.login(auth_info_1=username, auth_info_2=email, password=password)
#             client.save_cookies(cookies_file)
#             self.logger.info(f'{datetime.now()} - Cookies saved to {cookies_file}')
#             # twitterscraper_log.info(f'{datetime.now()} - Cookies saved to {cookies_file}')

#         tweet_count = 0
#         tweets = None

#         while tweet_count < self.minimum_tweets:
#             try:
#                 tweets = await self.get_tweets(client, tweets)
#             except TooManyRequests as e:
#                 rate_limit_reset = datetime.fromtimestamp(e.rate_limit_reset)
#                 self.logger.warning(f'{datetime.now()} - Rate limit reached. Waiting until {rate_limit_reset}')
#                 # twitterscraper_log.warning(f'{datetime.now()} - Rate limit reached. Waiting until {rate_limit_reset}')
#                 wait_time = rate_limit_reset - datetime.now()
#                 time.sleep(wait_time.total_seconds())
#                 continue

#             if not tweets:
#                 self.logger.info(f'{datetime.now()} - No more tweets found')
#                 # twitterscraper_log.info(f'{datetime.now()} - No more tweets found')
#                 break

#             for tweet in tweets:
#                 tweet_count += 1
#                 tweet_data = [
#                     tweet_count, tweet.user.name, tweet.text, tweet.created_at, 
#                     tweet.retweet_count, tweet.favorite_count
#                 ]

#                 with open('tweets.csv', 'a', newline='', encoding='utf-8') as file:
#                     writer = csv.writer(file)
#                     writer.writerow(tweet_data)

#             self.logger.info(f'{datetime.now()} - Got {tweet_count} tweets')
#             # twitterscraper_log.info(f'{datetime.now()} - Got {tweet_count} tweets')

#         self.logger.info(f'{datetime.now()} - Done! Got {tweet_count} tweets')
#         # twitterscraper_log.info(f'{datetime.now()} - Done! Got {tweet_count} tweets')


In [7]:
pwd

'c:\\Users\\OLLO\\3D Objects\\PROJET D.E\\pipeline_etl\\airflow\\plugins\\scrape_test'

In [6]:
import os
os.chdir('airflow/plugins/scrape_test/')


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'airflow/plugins/scrape_test/'

In [2]:
import os
os.chdir('airflow/plugins/scrape_test/')

In [3]:
import asyncio
from scraper import TwitterScraper
import asyncio
import nest_asyncio
nest_asyncio.apply()
scrape = TwitterScraper()

async def run_scraper():
    await scrape.main()

asyncio.run(run_scraper())


cookies_file: {'_twitter_sess': 'BAh7CSIKZmxhc2hJQzonQWN0aW9uQ29udHJvbGxlcjo6Rmxhc2g6OkZsYXNo%250ASGFzaHsABjoKQHVzZWR7ADoPY3JlYXRlZF9hdGwrCOGuhquSAToMY3NyZl9p%250AZCIlOGZhODcwMDE0MzcxNzEwYWM3Zjk2N2VhNzA0NWYwNDg6B2lkIiU4ZjVh%250AYjgwNjRmM2NlMzIxYjNiM2RkZjQ0ODMzZGU0MA%253D%253D--ca697796b15c8bb7a992e33fb35658228c641214', 'att': '1-oiD4A3V3rMsK4cgM4xfuF1IEmVZZ66340ahiWgzD', 'auth_token': '45905e32577bd3eac097e36dbd6700fe07643a1e', 'ct0': '4eb891e60d5aa2d797d72c569c869122', 'guest_id': 'v1%3A172945458275182349', 'guest_id_ads': 'v1%3A172945458275182349', 'guest_id_marketing': 'v1%3A172945458275182349', 'kdt': 'lDW1eu3M1zqOLG2rR7IxPjULKQ2eLtKdOQHsPt1H', 'personalization_id': '"v1_WJqxBj0FZVh6d6D+0moymQ=="', 'twid': '"u=1500831330300530694"'}
2024-11-03 17:44:21.154300 - Loading cookies from dictionary
Répertoire : c:\Users\OLLO\3D Objects\PROJET D.E\pipeline_etl\airflow\plugins\scrape_test
csv_file_path : hr-airflow/airflow/data\tweets.csv
2024-11-03 17:44:21.156296 - Getting tweets...
2024

In [13]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.Collecting pandas
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 1.0 MB/s eta 0:00:11
   -- ------------------------------------- 0.8/11.6 MB 1.0 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/11.6 MB 1.0 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/11.6 MB 1.0 MB/s eta 0:00:11
   ---- ----------------------------------- 1.3/11.6 MB 987.4 kB/s eta 0:00:11
   ----- ---------------------------------- 1.6/11.6 MB 999.0 kB/s eta 0:00:11
   ------ --------------------------------- 1.8/11.6 MB 1.0 MB/s eta 0:00:10
   ------- -------------------------------- 2.1/11.6 MB 1.0 MB/s eta 0:00:10
   -------- ------------------------------- 2.4/11.6 MB 1.0 MB/s eta 0:00:10
   -------

In [6]:
import os

In [1]:
import pandas as pd

In [8]:
os.chdir("airflow/data")

In [12]:
import polars as pl

In [21]:
tweets_df = pl.DataFrame(schema=['Tweet_count', 'Username', 'Text', 'Created At', 'Retweets', 'Likes'])
tweets_df

tweet_data={'Tweet_count': 1, 'Username': 'Elon Musk', 'Text': '@stillgray Meme analysis!', 'Created At': 'Tue Oct 29 11:05:22 +0000 2024', 'Retweets': 67, 'Likes': 890}
df1 = pl.DataFrame(tweet_data)
df1

tweets_df = tweets_df.vstack(df1)
tweets_df

SchemaError: type Int64 is incompatible with expected type Null

In [27]:
tweets_df = pl.DataFrame({
    'Tweet_count': [0],  # Valeur par défaut pour initialiser
    'Username': [''],    # Valeur par défaut pour initialiser
    'Text': [''],        # Valeur par défaut pour initialiser
    'Created At': [''],  # Valeur par défaut pour initialiser
    'Retweets': [0],     # Valeur par défaut pour initialiser
    'Likes': [0]         # Valeur par défaut pour initialiser
})
tweets_df

Tweet_count,Username,Text,Created At,Retweets,Likes
i64,str,str,str,i64,i64
0,"""""","""""","""""",0,0


In [23]:
import polars as pl

# Définir le DataFrame avec des valeurs initiales
tweets_df = pl.DataFrame({
    'Tweet_count': [0],  # Valeur par défaut pour initialiser
    'Username': [''],    # Valeur par défaut pour initialiser
    'Text': [''],        # Valeur par défaut pour initialiser
    'Created At': [''],  # Valeur par défaut pour initialiser
    'Retweets': [0],     # Valeur par défaut pour initialiser
    'Likes': [0]         # Valeur par défaut pour initialiser
})

# Créer le premier DataFrame avec des données
tweet_data = {
    'Tweet_count': 1,
    'Username': 'Elon Musk',
    'Text': '@stillgray Meme analysis!',
    'Created At': 'Tue Oct 29 11:05:22 +0000 2024',
    'Retweets': 67,
    'Likes': 890
}
df1 = pl.DataFrame(tweet_data)

# Ajouter df1 à tweets_df
tweets_df = tweets_df.vstack(df1)

# Afficher le DataFrame final
print(tweets_df)


shape: (2, 6)
┌─────────────┬───────────┬─────────────────┬────────────────────────────────┬──────────┬───────┐
│ Tweet_count ┆ Username  ┆ Text            ┆ Created At                     ┆ Retweets ┆ Likes │
│ ---         ┆ ---       ┆ ---             ┆ ---                            ┆ ---      ┆ ---   │
│ i64         ┆ str       ┆ str             ┆ str                            ┆ i64      ┆ i64   │
╞═════════════╪═══════════╪═════════════════╪════════════════════════════════╪══════════╪═══════╡
│ 0           ┆           ┆                 ┆                                ┆ 0        ┆ 0     │
│ 1           ┆ Elon Musk ┆ @stillgray Meme ┆ Tue Oct 29 11:05:22 +0000 2024 ┆ 67       ┆ 890   │
│             ┆           ┆ analysis!       ┆                                ┆          ┆       │
└─────────────┴───────────┴─────────────────┴────────────────────────────────┴──────────┴───────┘


In [26]:

type(tweets_df)

polars.dataframe.frame.DataFrame

In [24]:
tweets_df.write_csv("csv.csv")

SchemaError: type Int64 is incompatible with expected type Null

In [18]:
df1 = pl.DataFrame(
    {
        "foo": [1, 2],
        "bar": [6, 7],
        "ham": ["a", "b"],
    }
)
df2 = pl.DataFrame(
    {
        "foo": [3, 4],
        "bar": [8, 9],
        "ham": ["c", "d"],
    }
)
df1.vstack(df2)

foo,bar,ham
i64,i64,str
1,6,"""a"""
2,7,"""b"""
3,8,"""c"""
4,9,"""d"""


In [9]:

df = pd.read_csv("tweets.csv")

EmptyDataError: No columns to parse from file

In [1]:
import twikit

In [ ]:
import os
from twikit import Client

async def test_login():
    client = Client()
    username = os.environ.get("username")
    email = os.environ.get("password")
    password = os.environ.get("email")

    try:
        await client.login(auth_info_1=username, auth_info_2=email, password=password)
        print("Login successful!")
    except Exception as e:
        print(f"Login failed: {e}")

if __name__ == "__main__":
    import asyncio
    asyncio.run(test_login())


In [ ]:


















#     try:
#         # Connexion à la base de données PostgreSQL
#         connection = psycopg2.connect(
#             dbname=os.getenv('POSTGRES_DB'),
#             user=os.getenv('POSTGRES_USER'),
#             password=os.getenv('POSTGRES_PASSWORD'),
#             host='host.docker.internal',  # Utilisation directe de l'adresse hôte
#             port='5433'
#         )

#         cursor = connection.cursor()

#         # Création de la table si elle n'existe pas
#         create_table_query = sql.SQL("""
#             CREATE TABLE IF NOT EXISTS tweets (
#                 tweet_count SERIAL PRIMARY KEY,
#                 username VARCHAR(255),
#                 text TEXT,
#                 created_at TIMESTAMP,
#                 retweets INT,
#                 likes INT
#             )
#         """)
#         cursor.execute(create_table_query)

#         insert_query = sql.SQL("""
#             INSERT INTO tweets (tweet_count, username, text, created_at, retweets, likes)
#             VALUES (%s, %s, %s, %s, %s, %s)
#         """)

#         for tweet in tweets:
#             tweet_count += 1
#             tweet_data = (
#                 tweet_count,
#                 tweet.user.name,
#                 tweet.text,
#                 tweet.created_at,
#                 tweet.retweet_count,
#                 tweet.favorite_count
#             )
#             cursor.execute(insert_query, tweet_data)

#         connection.commit()
#         cursor.close()
#         connection.close()
#         print(f"{tweet_count} tweets have been inserted into the database.")
#     except Exception as e:
#         print(f"Error while inserting tweets into the database: {e}")



# class TwitterScraper:
#     MINIMUM_TWEETS = int(os.getenv('MINIMUM_TWEETS', 10)) 
#     QUERY = 'from:elonmusk lang:fr'
#     print(f"MINIMUM_TWEETS: {MINIMUM_TWEETS}, QUERY: {QUERY}")
#     def __init__(self):
#         self.tweet_count = 0
#         self.client = None
#         self.tweets_df = pl.DataFrame({
#             'Tweet_count': [0],  
#             'Username': [''],    
#             'Text': [''],        
#             'Created At': [''],  
#             'Retweets': [0],     
#             'Likes': [0]         
#         })

#     async def initialize_client(self):
#         self.client = Client(language='en-US')
        
#         cookies_file = {  
#             "_twitter_sess": "BAh7CSIKZmxhc2hJQzonQWN0aW9uQ29udHJvbGxlcjo6Rmxhc2g6OkZsYXNo%250ASGFzaHsABjoKQHVzZWR7ADoPY3JlYXRlZF9hdGwrCOGuhquSAToMY3NyZl9p%250AZCIlOGZhODcwMDE0MzcxNzEwYWM3Zjk2N2VhNzA0NWYwNDg6B2lkIiU4ZjVh%250AYjgwNjRmM2NlMzIxYjNiM2RkZjQ0ODMzZGU0MA%253D%253D--ca697796b15c8bb7a992e33fb35658228c641214",
#             "att": "1-oiD4A3V3rMsK4cgM4xfuF1IEmVZZ66340ahiWgzD",
#             "auth_token": "45905e32577bd3eac097e36dbd6700fe07643a1e",
#             "ct0": "4eb891e60d5aa2d797d72c569c869122",
#             "guest_id": "v1%3A172945458275182349",
#             "guest_id_ads": "v1%3A172945458275182349",
#             "guest_id_marketing": "v1%3A172945458275182349",
#             "kdt": "lDW1eu3M1zqOLG2rR7IxPjULKQ2eLtKdOQHsPt1H",
#             "personalization_id": "\"v1_WJqxBj0FZVh6d6D+0moymQ==\"",
#             "twid": "\"u=1500831330300530694\""
#         }
        
#         print(f'cookies_file: {cookies_file}')

#         # Chargement des cookies à partir du dictionnaire
#         if cookies_file:
#             print(f'{datetime.now()} - Loading cookies from dictionary')
#             self.client.set_cookies(cookies_file)
#         else:
#             print(f'{datetime.now()} - No cookies found. Please log in to generate cookies.')

#         # Créez le répertoire s'il n'existe pas
#         os.makedirs(data_path, exist_ok=True)
#         print(f"Répertoire : {os.getcwd()}")

#         self.csv_file_path = os.path.join(data_path, 'tweets.csv')
#         print(f"csv_file_path : {self.csv_file_path}")
#         print("#########################################################################################")
#         print(f"self_client: {self.client}")

#     async def get_tweets(self, tweets=None):
#         if tweets is None:
#             print(f'{datetime.now()} - Getting tweets...')
#             try:
#                 tweets = await self.client.search_tweet(self.QUERY, product='Top')
#             except Exception as e:
#                 print(f'{datetime.now()} - Error getting tweets: {e}')
#                 return None
#         else:
#             wait_time = randint(5, 10)
#             print(f'{datetime.now()} - Getting next tweets after {wait_time} seconds...')
#             await asyncio.sleep(wait_time)
#             try:
#                 tweets = await tweets.next()
#             except Exception as e:
#                 print(f'{datetime.now()} - Error getting next tweets: {e}')
#                 return None

#         if tweets:
#             print(f'{datetime.now()} - Retrieved {len(tweets)} tweets')
#         else:
#             print(f'{datetime.now()} - No tweets found for query.')
#         return tweets

#     async def save_tweets(self, tweets):
#         await self.insert_tweets_to_db(tweets)  # Insérer les tweets dans la base de données
#     async def insert_tweets_to_db(self, tweets):
#         try:
#             # Connexion à la base de données PostgreSQL
#             connection = psycopg2.connect(
#                 dbname=os.getenv('POSTGRES_DB'),
#                 user=os.getenv('POSTGRES_USER'),
#                 password=os.getenv('POSTGRES_PASSWORD'),
#                 host='host.docker.internal',  # Utilisation directe de l'adresse hôte
#                 port='5433'
#             )

#             cursor = connection.cursor()

#             # Création de la table si elle n'existe pas
#             create_table_query = sql.SQL("""
#                 CREATE TABLE IF NOT EXISTS tweets (
#                     tweet_count SERIAL PRIMARY KEY,
#                     username VARCHAR(255),
#                     text TEXT,
#                     created_at TIMESTAMP,
#                     retweets INT,
#                     likes INT
#                 )
#             """)
#             cursor.execute(create_table_query)

#             insert_query = sql.SQL("""
#                 INSERT INTO tweets (tweet_count, username, text, created_at, retweets, likes)
#                 VALUES (%s, %s, %s, %s, %s, %s)
#             """)

#             for tweet in tweets:
#                 self.tweet_count += 1
#                 tweet_data = (
#                     self.tweet_count,
#                     tweet.user.name,
#                     tweet.text,
#                     tweet.created_at,
#                     tweet.retweet_count,
#                     tweet.favorite_count
#                 )
#                 cursor.execute(insert_query, tweet_data)

#             connection.commit()
#             cursor.close()
#             connection.close()
#             print(f"{self.tweet_count} tweets have been inserted into the database.")
#         except Exception as e:
#             print(f"Error while inserting tweets into the database: {e}")


#     async def write_to_csv(self):
#         # Écrit le DataFrame dans un fichier CSV
#         print(f'tweets_df__tweets_df:{self.tweets_df}')
#         self.tweets_df.write_csv(self.csv_file_path)
#         print("save successful")

#     async def main(self):
#         await self.initialize_client()
#         tweets = None

#         while self.tweet_count < self.MINIMUM_TWEETS:
#             try:
#                 tweets = await self.get_tweets(tweets)
#             except TooManyRequests as e:
#                 rate_limit_reset = datetime.fromtimestamp(e.rate_limit_reset)
#                 print(f'{datetime.now()} - Rate limit reached. Waiting until {rate_limit_reset}')
#                 wait_time = rate_limit_reset - datetime.now()
#                 await asyncio.sleep(wait_time.total_seconds())
#                 continue
#             except Exception as e:
#                 print(f'{datetime.now()} - An error occurred: {e}')
#                 break

#             if not tweets:
#                 print(f'{datetime.now()} - No more tweets found')
#                 break
            
#             await self.save_tweets(tweets)

#         await self.write_to_csv()  # Écriture finale dans le CSV
#         print(f'{datetime.now()} - Done! Got {self.tweet_count} tweets found')

In [1]:
cookies_file = {
    "_twitter_sess": "BAh7CSIKZmxhc2hJQzonQWN0aW9uQ29udHJvbGxlcjo6Rmxhc2g6OkZsYXNo%250ASGFzaHsABjoKQHVzZWR7ADoPY3JlYXRlZF9hdGwrCOGuhquSAToMY3NyZl9p%250AZCIlOGZhODcwMDE0MzcxNzEwYWM3Zjk2N2VhNzA0NWYwNDg6B2lkIiU4ZjVh%250AYjgwNjRmM2NlMzIxYjNiM2RkZjQ0ODMzZGU0MA%253D%253D--ca697796b15c8bb7a992e33fb35658228c641214",
    "att": "1-oiD4A3V3rMsK4cgM4xfuF1IEmVZZ66340ahiWgzD",
    "auth_token": "45905e32577bd3eac097e36dbd6700fe07643a1e",
    "ct0": "4eb891e60d5aa2d797d72c569c869122",
    "guest_id": "v1%3A172945458275182349",
    "guest_id_ads": "v1%3A172945458275182349",
    "guest_id_marketing": "v1%3A172945458275182349",
    "kdt": "lDW1eu3M1zqOLG2rR7IxPjULKQ2eLtKdOQHsPt1H",
    "personalization_id": "\"v1_WJqxBj0FZVh6d6D+0moymQ==\"",
    "twid": "\"u=1500831330300530694\""
}

In [4]:
from twikit import Client, TooManyRequests

try:
    print("client")
    client = Client(language='en-US')
    print('Loading cookies from dictionary')
    client.set_cookies(cookies_file)
    print("client_client:",client)
except Exception as e:
    print(f"error: {e}")

client
Loading cookies from dictionary
client_client: <twikit.client.client.Client object at 0x0000018438C1D8D0>


In [6]:
def cal():
    1+1

def sous():
    cal

sous()

In [ ]:
def cal

In [ ]:
def initialize_client(self):
    client = Client(language='en-US')
    
    cookies_file = {  
        "_twitter_sess": "BAh7CSIKZmxhc2hJQzonQWN0aW9uQ29udHJvbGxlcjo6Rmxhc2g6OkZsYXNo%250ASGFzaHsABjoKQHVzZWR7ADoPY3JlYXRlZF9hdGwrCOGuhquSAToMY3NyZl9p%250AZCIlOGZhODcwMDE0MzcxNzEwYWM3Zjk2N2VhNzA0NWYwNDg6B2lkIiU4ZjVh%250AYjgwNjRmM2NlMzIxYjNiM2RkZjQ0ODMzZGU0MA%253D%253D--ca697796b15c8bb7a992e33fb35658228c641214",
        "att": "1-oiD4A3V3rMsK4cgM4xfuF1IEmVZZ66340ahiWgzD",
        "auth_token": "45905e32577bd3eac097e36dbd6700fe07643a1e",
        "ct0": "4eb891e60d5aa2d797d72c569c869122",
        "guest_id": "v1%3A172945458275182349",
        "guest_id_ads": "v1%3A172945458275182349",
        "guest_id_marketing": "v1%3A172945458275182349",
        "kdt": "lDW1eu3M1zqOLG2rR7IxPjULKQ2eLtKdOQHsPt1H",
        "personalization_id": "\"v1_WJqxBj0FZVh6d6D+0moymQ==\"",
        "twid": "\"u=1500831330300530694\""
    }
    
    print(f'cookies_file: {cookies_file}')

    # Chargement des cookies à partir du dictionnaire
    if cookies_file:
        print(f'{datetime.now()} - Loading cookies from dictionary')
        self.client.set_cookies(cookies_file)
    else:
        print(f'{datetime.now()} - No cookies found. Please log in to generate cookies.')

In [7]:
import re
import nltk
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# nltk.download('punkt_tab')
# Définir le chemin de données NLTK personnalisé (optionnel)
nltk.data.path.append(os.path.join(os.path.expanduser("~"), "nltk_data"))

# Fonction pour vérifier et télécharger les ressources NLTK nécessaires
def ensure_nltk_resources():
    """
    Vérifie et télécharge les ressources NLTK nécessaires.
    """
    required_resources = ['stopwords', 'punkt', 'wordnet', 'omw-1.4','punkt_tab']
    for resource in required_resources:
        try:
            if not nltk.data.find(f'tokenizers/{resource}' if resource == 'punkt' 
                                else f'corpora/{resource}'):
                nltk.download(resource)
        except LookupError:
            nltk.download(resource)

# S'assurer que les ressources sont disponibles
ensure_nltk_resources()

# Initialiser les ressources NLTK
try:
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
except LookupError as e:
    print(f"Erreur lors du chargement des ressources NLTK: {e}")
    print("Tentative de re-téléchargement des ressources...")
    ensure_nltk_resources()
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

def clean_text(text):
    """
    Nettoie et pré-traite une phrase pour des tâches de NLP en utilisant nltk.
    
    Args:
    - text (str): La phrase à nettoyer.
    
    Returns:
    - str: La phrase nettoyée.
    """
    try:
        # Vérifier que l'entrée est une chaîne de caractères
        if not isinstance(text, str):
            raise ValueError("L'entrée doit être une chaîne de caractères")
            
        # Convertir en minuscules
        text = text.lower()
        
        # Suppression des URL et emails
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        
        # Suppression des chiffres
        text = re.sub(r'\d+', '', text)
        
        # Suppression de la ponctuation
        text = re.sub(r'[^\w\s]', '', text)
        
        # Tokenisation
        try:
            words = word_tokenize(text)
        except LookupError:
            print("Erreur de tokenisation, re-téléchargement des ressources...")
            nltk.download('punkt')
            words = word_tokenize(text)
        
        # Suppression des stopwords et lemmatisation
        cleaned_text = ' '.join([
            lemmatizer.lemmatize(word) 
            for word in words 
            if word not in stop_words and word.isalpha()
        ])
        
        return cleaned_text
    
    except Exception as e:
        print(f"Une erreur s'est produite lors du nettoyage du texte: {str(e)}")
        return text  # Retourner le texte original en cas d'erreur

# Exemple d'utilisation
try:
    phrase = "This is an example sentence, with numbers like 123 and an email address test@example.com."
    cleaned = clean_text(phrase)
    print("Texte original:", phrase)
    print("Texte nettoyé:", cleaned)
except Exception as e:
    print(f"Erreur lors du traitement: {str(e)}") 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\OLLO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\OLLO\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\OLLO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Texte original: This is an example sentence, with numbers like 123 and an email address test@example.com.
Texte nettoyé: example sentence number like email address


In [1]:
import polars as pl

In [3]:
pwd

'c:\\Users\\OLLO\\3D Objects\\PROJET D.E\\pipeline_etl'

In [5]:
data = pl.read_csv("airflow/plugins/scrape_test/hr-airflow/airflow/data/tweets.csv")

In [6]:
data

Tweet_count,Username,Text,Created At,Retweets,Likes
i64,str,str,str,i64,i64
0,"""""","""""","""""",0,0
1,"""Elon Musk""","""Liberté Liberté! Liberté?""","""Sun Aug 25 05:07:03 +0000 2024""",44337,347743
2,"""Elon Musk""","""🇫🇷🇫🇷🇫🇷 Vive la France! 🇫🇷🇫🇷🇫🇷 …","""Fri Jul 26 13:39:24 +0000 2024""",20129,229080
3,"""Elon Musk""","""@waitbutwhy Earth has a thin c…","""Mon Oct 28 18:57:22 +0000 2024""",72,994
4,"""Elon Musk""","""Fraud""","""Tue Oct 29 11:29:54 +0000 2024""",74378,234555
…,…,…,…,…,…
16,"""Elon Musk""","""Dangerous times""","""Sun Aug 25 03:41:34 +0000 2024""",74407,311130
17,"""Elon Musk""","""@OptimoPrincipi @SoterioPoliti…","""Tue Jul 09 22:14:34 +0000 2024""",40,764
18,"""Elon Musk""","""@wallstreetbets @MikeBenzCyber…","""Sat May 25 05:04:44 +0000 2024""",46,560


In [8]:
df_cleaned = data.with_columns(
    pl.col("Text").map_elements(clean_text).alias("Text")
)
df_cleaned

C:\Users\OLLO\AppData\Local\Temp\ipykernel_3780\4290007796.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df_cleaned = data.with_columns(


Tweet_count,Username,Text,Created At,Retweets,Likes
i64,str,str,str,i64,i64
0,"""""","""""","""""",0,0
1,"""Elon Musk""","""liberté liberté liberté""","""Sun Aug 25 05:07:03 +0000 2024""",44337,347743
2,"""Elon Musk""","""vive la france ville de lumièr…","""Fri Jul 26 13:39:24 +0000 2024""",20129,229080
3,"""Elon Musk""","""waitbutwhy earth thin crust so…","""Mon Oct 28 18:57:22 +0000 2024""",72,994
4,"""Elon Musk""","""fraud""","""Tue Oct 29 11:29:54 +0000 2024""",74378,234555
…,…,…,…,…,…
16,"""Elon Musk""","""dangerous time""","""Sun Aug 25 03:41:34 +0000 2024""",74407,311130
17,"""Elon Musk""","""optimoprincipi soteriopolitics…","""Tue Jul 09 22:14:34 +0000 2024""",40,764
18,"""Elon Musk""","""wallstreetbets mikebenzcyber d…","""Sat May 25 05:04:44 +0000 2024""",46,560
